# LLM Powered Car-Buying Assistant

With over 40 automotive brands sold in the US, the car market is a dynamic landscape. Individuals can make informed purchasing decisions by parsing through reviews on the internet, but the process is time-consuming.

This LLM application provides users an immediate analysis of the current best cars at a user specified price point.

In [3]:
import os
import openai

from metaphor_python import Metaphor

openai.api_key = os.getenv("OPENAI_API_KEY")
metaphor = Metaphor(os.getenv("METAPHOR_API_KEY"))

# Approach without Metaphor

Let's assume our user wants to purchase a new EV with $50k.

The results here are somewhat useful, but the OpenAI model is unable to access real-time information on the latest car models.

In [8]:
USER_QUESTION = "What are the best electric cars from 2023 that are around $50k?"

SYSTEM_MESSAGE_SEARCH = "You are a helpful assistant."

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE_SEARCH},
        {"role": "user", "content": USER_QUESTION},
    ],
)

print(completion.choices[0].message.content)

As an AI language model, I don't have real-time information on specific car models from 2023. However, I can provide you with some electric vehicles that were popular in 2022 and may fall within the $50,000 price range. It's important to note that prices can vary depending on specifications, location, and available incentives. Here are a few options:

1. Tesla Model 3: The base model starts at around $40,000, offering good range and performance.
2. Chevrolet Bolt EV: A compact electric car with a starting price around $36,000, providing decent range and features.
3. Nissan Leaf Plus: The Leaf Plus offers an extended range and starts at approximately $40,000.
4. Hyundai Kona Electric: This affordable electric SUV starts around $40,000 and offers competitive range.
5. Ford Mustang Mach-E: Ford's all-electric SUV offers various trims and starts around $43,000 before incentives.

Keep in mind that availability and pricing can change significantly from year to year, so these suggestions may

# Approach with Metaphor

We can use Metaphor's neural search engine to better address the user's question.

## Step 1: Generate search queries

Using OpenAI's API, we can generate a search query for the user's question.

In [5]:
SYSTEM_MESSAGE_SEARCH = "You are a helpful assistant that generates search queries based on user questions. Only generate one search query."

search_completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE_SEARCH},
        {"role": "user", "content": USER_QUESTION},
    ],
)

search_query = search_completion.choices[0].message.content

print(search_query)

"Top-rated electric cars 2023 under $50k"


## Step 2: Neural search using query

We can use Metaphor's neural search engine to generate relevant results for our search query. To filter the large volume of websites, we specify a set of highly trusted and active car review domains.

Note: The number of results is restricted to 4 to fit the token limitations of the OpenAI model in Step 3. Users with access to higher OpenAI models may increase this limit to produce more robust results.

In [6]:
valid_domains = ["caranddriver.com", "motortrend.com", "edmunds.com", "consumerreports.com", "kbb.com", "jdpower.com"]

search_response = metaphor.search(
    search_query,
    num_results=4,
    include_domains=valid_domains,
    start_published_date="2022-01-01",
    use_autoprompt=True,
)

result_content = search_response.get_contents().contents

## Step 3: Summarize search results

Leveraging Metaphor's content retrieval API, we can aggregate search results for our query and then use OpenAI's LLM to summarize the results. Compared to the initial results without Metaphor, our program is able to suggest new cars from 2023 and provide exact price points.

In [7]:
SYSTEM_MESSAGE_SUMMARIZE = "You are a helpful car-buying assistant. Use the user's input to suggest a list of new cars."

result_query = f"Query: {search_query}\n"
for j, extracted_result in enumerate([result.extract for result in result_content]):
    result_query += f"Article {j}: {extracted_result}\n"

summarization_completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE_SUMMARIZE},
        {"role": "user", "content": result_query},
    ],
)

summary = summarization_completion.choices[0].message.content

print(summary)

Based on your query for "Top-rated electric cars 2023 under $50k," here is a list of new cars that fit your criteria:

1. 2023 Mini Cooper Electric: This go-kart-like EV has a starting price of just over $30,000 and offers a range of 110 miles. It comes with lots of standard features and provides a fun driving experience.

2. 2023 Nissan Ariya: Nissan's new electric SUV offers up to 304 miles of driving range and has a starting price of $44,485. It features a sporty design, advanced driver-assistance features, and infotainment technology.

3. 2024 Chevrolet Equinox EV: This compact SUV is expected to have a starting price of around $30,000 and a range of up to 300 miles. It features a sleek design, a spacious interior, and comes with advanced technology options.

4. 2023 Kia Niro EV: The redesigned Niro EV offers a range of up to 253 miles and has a starting price that fits within your budget. It comes with a distinctive design, recycled materials in the interior, and a 10.3-inch infot